In [1]:
import pandas as pd
import numpy as np 
import re
import nltk
import multiprocessing
from pathlib import Path

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from IPython.core.interactiveshell import InteractiveShell
from pathlib import Path
from scipy import spatial


In [3]:
pd.set_option("display.max_columns", None)
InteractiveShell.ast_node_interactivity = "all"
nltk.download('stopwords') 
nltk.download('punkt')
cores = multiprocessing.cpu_count()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fourz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fourz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
class ModelWord2Vec:
    """
    Класс для обучения и использования модели Word2Vec.

    Аргументы:
        final_df (pandas.DataFrame): pandas DataFrame, содержащий данные для обучения модели.

    Атрибуты:
        vectors (list): Список векторов, представляющих слова в DataFrame final_df.
        model (gensim.models.Word2Vec): Обученная модель Word2Vec.

    Методы:
        create_model(self): Обучает модель Word2Vec.
        vec_df(self): Преобразует DataFrame final_df в DataFrame, содержащий векторы для каждого слова.
    """
    def __init__(self, final_df: pd.DataFrame):
        self.vectors = []
        self.final_df = final_df
        self.model = None

    def create_model(self):
        """
        Обучает модель Word2Vec.

        Возвращает:
            gensim.models.Word2Vec: Обученная модель Word2Vec.
        """
        self.model = Word2Vec(
            self.final_df["synopsis_tokens"],
            window=6,
            vector_size=50,
            alpha=0.03,
            min_alpha=0.0007,
            negative=20,
            workers=cores - 1,
            sg=0
        )
        self.model.train(
            self.final_df["synopsis_tokens"],
            total_examples=len(self.final_df["synopsis_tokens"]),
            epochs=1
        )
        return self.model

    def vec_df(self):
        """
        Преобразует DataFrame final_df в DataFrame, содержащий векторы для каждого слова.

        Возвращает:
            pandas.DataFrame: DataFrame, содержащий векторы для каждого слова.
        """
        for synopsis_tokens in self.final_df["synopsis_tokens"]:
            synopsis_vector = np.zeros(self.model.vector_size)
            num_tokens = 0
            for token in synopsis_tokens:
                if token in self.model.wv.key_to_index:
                    vector = self.model.wv.get_vector(token)
                    synopsis_vector += vector
                    num_tokens += 1
            if num_tokens > 0:
                synopsis_vector /= num_tokens
                self.vectors.append(synopsis_vector)
        self.final_df["vectors"] = self.vectors
        
        return self.final_df

    def predict(self, synopsis: str):
        """
        Предсказывает название серии для данного синопсиса.

        Аргументы:
            synopsis: Синопсис серии.

        Возвращает:
            Предсказанное название серии.
        """
        synopsis = preprocessor_text(synopsis) 

        lst = set([i for i in synopsis.split() if i in self.model.wv.key_to_index])
        
        # Получение векторов слов в синопсисе.
        context_word_vectors = [self.model.wv[word] for word in lst]

        # Расчет среднего вектора слов для синопсиса.
        predicted_vector = np.mean(context_word_vectors, axis=0)

        # Расчет сходства между предсказанным вектором и векторами в наших данных
        similarities = df['vectors'].map(lambda x: 1 - spatial.distance.cosine(np.array(x, dtype='float16'), predicted_vector.astype('float16')))

        # сортируем топ самую малую разницу
        most_similar_index = similarities.argmax()
        
        return df.iloc[similarities.argmax()]


def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    """
    Аргументы:
        df: Фрейм данных.

    Возвращает:
        df + токенизированные колонки.
    """
    df.dropna(subset=['release_year', 'rating', 'cast'], inplace=True)
    df[['release_year']] = df[['release_year']].astype(int)
    df["synopsis_tokens"] = df["synopsis1"].apply(word_tokenize)
#     df['series_title'] = df['series_title'].drop_duplicates().dropna()
    
    return df


def preprocessor_text(text: str) -> str:
    """
    Нормализует текстовую строку.

    Аргументы:
        text: Текстовая строка для нормализации.

    Возвращает:
        Нормализованную текстовую строку.
    """
    text = re.sub('<[^>]*>', '', text)

    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + \
        ' '.join(emoticons).replace('-', '')

    tokens = word_tokenize(text)

    # удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    processed_text = ' '.join(filtered_tokens)

    return processed_text


if __name__ == "__main__":
    df = pd.read_csv(
        "C:/Users/fourz/OneDrive/Рабочий стол/Word2Vec/filter_data.csv",
        usecols=lambda column: column != 'Unnamed: 0' and column != 'synopsis_tokens'
    )
    synaps = input()

    final_df = preprocess(df)

    pp = preprocessor_text(synaps)

    model_word2vec = ModelWord2Vec(final_df)
    model = model_word2vec.create_model()

    emb = model_word2vec.vec_df()
    
    pred = model_word2vec.predict(pp)
    
    display(pred[['series_title', 'release_year', 'runtime', 'genre', 'rating', 'cast', 'synopsis', 'end_year']])


crime in city 


series_title                                         The Outsider
release_year                                                 2020
runtime                                                      60.0
genre                                       Crime, Drama, Fantasy
rating                                                        7.7
cast            Ben Mendelsohn, Bill Camp, Jeremy Bobb, Mare W...
synopsis        Investigators are confounded over an unspeakab...
end_year                                                   2020.0
Name: 517, dtype: object